In [1]:
import warnings

warnings.simplefilter("ignore")

In [2]:
#%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 81.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 22.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
from azureml.core import Workspace
ws = Workspace.from_config('')

In [9]:
import mlflow

mlflow.set_experiment(experiment_name="cars-mlr2")

<Experiment: artifact_location='', creation_time=1671823866008, experiment_id='3edbaace-d76b-4190-910e-c554d993d932', last_update_time=None, lifecycle_stage='active', name='cars-mlr2', tags={}>

In [10]:
import pandas as pd
file_url = r"Cars.csv"
df = pd.read_csv(file_url)

In [11]:
X = df.iloc[:, 2:6].values
y = df.iloc[:, 0].values

In [12]:
mlflow.sklearn.autolog()

2022/12/24 00:16:30 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


In [13]:
run = mlflow.start_run()

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
from sklearn.linear_model import LinearRegression
multilinear = LinearRegression()
multilinear.fit(X_train,y_train)

LinearRegression()

In [16]:
y_pred = multilinear.predict(X_test)

In [17]:
from sklearn.metrics import r2_score
accuracy = r2_score(y_test, y_pred)
accuracy

0.7105173175987358

In [18]:
mlflow.end_run()

In [19]:
run = mlflow.get_run(run.info.run_id)

In [20]:
pd.DataFrame(data=[run.data.params], index=["Value"]).T

,Value
copy_X,True
fit_intercept,True
n_jobs,None
positive,False


In [21]:
pd.DataFrame(data=[run.data.metrics], index=["Value"]).T

,Value
training_r2_score,0.766595
training_score,0.766595
training_rmse,2.930500
training_mse,8.587831
training_mae,1.945283
r2_score_X_test,0.710517


In [22]:
client = mlflow.tracking.MlflowClient()

In [23]:
client.list_artifacts(run_id=run.info.run_id)

[<FileInfo: file_size=-1, is_dir=False, path='estimator.html'>,
 <FileInfo: file_size=-1, is_dir=False, path='metric_info.json'>,
 <FileInfo: file_size=-1, is_dir=True, path='model'>]

In [24]:
regressor = mlflow.sklearn.load_model(f"runs:/{run.info.run_id}/model")

In [25]:
regressor.predict(X_test)

array([24.84303264, 28.24331711, 34.4505337 , 25.29971213, 32.09981075,
       33.79268187, 40.47333476, 35.04828473, 40.47333476, 36.89489481,
       23.6246313 , 38.11835714, 24.84303264, 24.21167565, 42.06180763,
       35.90252503, 30.20871483, 40.48392591, 30.20871483, 30.20871483,
       24.84303264, 35.94422557, 36.2784823 , 24.84303264, 23.36939651,
       34.2751037 , 37.197936  , 32.84912834, 32.11507866, 26.70149115,
       35.34257263, 36.3682537 , 24.84303264, 38.11835714, 40.47333476,
       35.81418338, 35.04385508, 37.97395098, 43.05310041, 34.17629951,
       43.71865778, 36.93280606, 37.32021288, 37.61534547, 36.3682537 ,
       28.87368387, 35.94422557, 34.96167709, 34.17629951, 19.51700417,
       36.3682537 , 14.69329161, 36.54883285, 36.89489481, 35.81418338,
       40.48392591, 32.27137381, 32.09981075, 37.67774868, 24.2078474 ,
       32.11507866, 37.197936  , 37.32021288, 34.73215607, 26.70149115,
       37.61534547, 18.79076337, 36.54883285, 32.60776465, 35.04

In [26]:
from sklearn.pipeline import Pipeline

In [27]:
pipeline=Pipeline(steps=[("model",multilinear)])

In [28]:
pipeline

Pipeline(steps=[('model', LinearRegression())])

In [29]:
from mlflow.models import infer_signature

signature = infer_signature(X_test, y_test)

In [30]:
signature

inputs: 
  [Tensor('float64', (-1, 4))]
outputs: 
  [Tensor('float64', (-1,))]

In [31]:
with mlflow.start_run() as run:
    pipeline.fit(X_train, y_train)
    mlflow.sklearn.log_model(pipeline, artifact_path="pipeline", signature=signature)

In [32]:
pipeline_model = mlflow.sklearn.load_model(f"runs:/{run.info.run_id}/pipeline")

In [33]:
pipeline_model

Pipeline(steps=[('model', LinearRegression())])

In [34]:
pipeline_model.predict(X_test)

array([24.84303264, 28.24331711, 34.4505337 , 25.29971213, 32.09981075,
       33.79268187, 40.47333476, 35.04828473, 40.47333476, 36.89489481,
       23.6246313 , 38.11835714, 24.84303264, 24.21167565, 42.06180763,
       35.90252503, 30.20871483, 40.48392591, 30.20871483, 30.20871483,
       24.84303264, 35.94422557, 36.2784823 , 24.84303264, 23.36939651,
       34.2751037 , 37.197936  , 32.84912834, 32.11507866, 26.70149115,
       35.34257263, 36.3682537 , 24.84303264, 38.11835714, 40.47333476,
       35.81418338, 35.04385508, 37.97395098, 43.05310041, 34.17629951,
       43.71865778, 36.93280606, 37.32021288, 37.61534547, 36.3682537 ,
       28.87368387, 35.94422557, 34.96167709, 34.17629951, 19.51700417,
       36.3682537 , 14.69329161, 36.54883285, 36.89489481, 35.81418338,
       40.48392591, 32.27137381, 32.09981075, 37.67774868, 24.2078474 ,
       32.11507866, 37.197936  , 37.32021288, 34.73215607, 26.70149115,
       37.61534547, 18.79076337, 36.54883285, 32.60776465, 35.04

Bad pipe message: %s [b'nY\xba\x91E\xf7\xbe\xca\x08O\xa0\x89\x97W5\x7f\xcb\xd5 !\x98\x14\x85Nn\xd5\xc47\x1e|Z\xcb~\xcd\x16\xf4\x19\xa1\xf2\xec\x17\xdf\xb0\x06X\xf8\xd9\xab\r\xd9"\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00']
Bad pipe message: %s [b'\x1d\x00 \xaa\xdf\xd8\xea`\x93\xbb\xce\xf5\xfaLO"\xa9R\xa1z\xad\xdd\xcfm\xb8\xbe\x1bu\xc1\x81\x07\xd7\xe6S\x13']
Bad pipe message: %s [b'7\x14\x9f\xc4\x99-\xac\x98\xe4\x00G\xd0\xa2\x0e]\xa6\xc1\xbc [b\xbc\x07y\x16\xe8\x11\xbd\xe52\x12\xf2>\xbf}?h\xf3\xf5\xd9]\xbbJtk\xb7{\xa22pR\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t1